# 🎮 Single Game: Red Team vs Blue Team

This notebook walks through running a single adversarial game where:
- **Red Team** generates vulnerable IaC code with hidden security flaws
- **Blue Team** attempts to detect all vulnerabilities
- **Judge** scores the match using precision, recall, and F1 metrics

## Prerequisites
- AWS credentials configured (for Bedrock access)
- `.env` file with `AWS_REGION` set

## 1. Setup and Imports

In [ ]:
# Add project root to path
import sys
from pathlib import Path

# Navigate to project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# Core imports
import asyncio
import json
import os
from datetime import datetime

# Project imports
from src.agents.red_team_agent import create_red_team_agent, Difficulty
from src.agents.blue_team_agent import create_blue_team_agent, DetectionMode
from src.agents.judge_agent import JudgeAgent, score_results_to_dict
from src.game.scenarios import ScenarioGenerator

print(f"✓ Project root: {project_root}")
print("✓ All imports successful!")

## 2. Configure the Game

Modify these settings to customize the adversarial match:

In [ ]:
# ============================================
# 🎮 GAME CONFIGURATION - Modify these!
# ============================================

# The scenario describes what infrastructure to create
SCENARIO = "Create an S3 bucket for storing healthcare PHI data with proper security controls"

# Difficulty affects how subtle the vulnerabilities are
# Options: "easy", "medium", "hard"
DIFFICULTY = "medium"

# Models to use (must be available in your AWS Bedrock region)
RED_TEAM_MODEL = "us.anthropic.claude-3-5-haiku-20241022-v1:0"   # Attacker
BLUE_TEAM_MODEL = "us.anthropic.claude-3-5-haiku-20241022-v1:0"  # Defender

# IaC settings
LANGUAGE = "terraform"      # or "cloudformation"
CLOUD_PROVIDER = "aws"      # or "azure", "gcp"

# AWS region for Bedrock
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

print(f"📋 Scenario: {SCENARIO}")
print(f"🎯 Difficulty: {DIFFICULTY}")
print(f"🔴 Red Team Model: {RED_TEAM_MODEL}")
print(f"🔵 Blue Team Model: {BLUE_TEAM_MODEL}")

## 3. 🔴 Red Team Attack

The Red Team generates IaC code that appears legitimate but contains hidden vulnerabilities.

In [ ]:
# Create Red Team agent
red_agent = create_red_team_agent(
    model_id=RED_TEAM_MODEL,
    region=AWS_REGION,
    difficulty=Difficulty[DIFFICULTY.upper()],
    language=LANGUAGE,
    cloud_provider=CLOUD_PROVIDER
)

print("🔴 Red Team agent created")
print(f"   Difficulty: {DIFFICULTY}")
print(f"   Language: {LANGUAGE}")

In [ ]:
# Generate vulnerable IaC
print("🔴 Red Team generating vulnerable code...")
red_result = await red_agent.generate(SCENARIO)

print(f"\n✓ Generated {len(red_result.files)} file(s)")
print(f"✓ Injected {len(red_result.vulnerabilities)} vulnerabilities")

# Show the generated code
for filename, content in red_result.files.items():
    print(f"\n📄 {filename}:")
    print("-" * 50)
    print(content[:1500] + "..." if len(content) > 1500 else content)

In [ ]:
# Show what Red Team secretly injected (ground truth)
print("🔴 RED TEAM'S SECRET VULNERABILITIES (Ground Truth):")
print("=" * 60)

for i, vuln in enumerate(red_result.vulnerabilities, 1):
    print(f"\n{i}. {vuln.get('title', 'Unknown')}")
    print(f"   Type: {vuln.get('vulnerability_type', 'N/A')}")
    print(f"   Resource: {vuln.get('resource_name', 'N/A')}")
    print(f"   Severity: {vuln.get('severity', 'N/A')}")

## 4. 🔵 Blue Team Defense

The Blue Team analyzes the code and attempts to find all vulnerabilities.

In [ ]:
# Create Blue Team agent
blue_agent = create_blue_team_agent(
    model_id=BLUE_TEAM_MODEL,
    region=AWS_REGION,
    mode=DetectionMode.LLM_ONLY  # Options: LLM_ONLY, TOOLS_ONLY, HYBRID
)

print("🔵 Blue Team agent created")
print(f"   Mode: LLM_ONLY")

In [ ]:
# Blue Team analyzes the code
print("🔵 Blue Team analyzing code...")
blue_result = await blue_agent.analyze(red_result.files)

print(f"\n✓ Blue Team found {len(blue_result.findings)} potential vulnerabilities")

# Show Blue Team's findings
print("\n🔵 BLUE TEAM'S FINDINGS:")
print("=" * 60)

for i, finding in enumerate(blue_result.findings, 1):
    print(f"\n{i}. {finding.get('title', 'Unknown')}")
    print(f"   Resource: {finding.get('resource_name', 'N/A')}")
    print(f"   Severity: {finding.get('severity', 'N/A')}")
    print(f"   Evidence: {finding.get('evidence', 'N/A')[:100]}...")

## 5. ⚖️ Judge Scoring

The Judge compares Red Team's ground truth against Blue Team's findings to calculate metrics.

In [ ]:
# Judge scores the match
judge = JudgeAgent()
scoring = judge.score(
    red_vulnerabilities=red_result.vulnerabilities,
    blue_findings=blue_result.findings
)

# Convert to dict for display
results = score_results_to_dict(scoring)

print("⚖️ JUDGE'S SCORING RESULTS")
print("=" * 60)
print(f"\n📊 METRICS:")
print(f"   Precision:    {results['metrics']['precision']:.1%}")
print(f"   Recall:       {results['metrics']['recall']:.1%}")
print(f"   F1 Score:     {results['metrics']['f1_score']:.1%}")
print(f"   Evasion Rate: {results['metrics']['evasion_rate']:.1%}")

print(f"\n📈 COUNTS:")
print(f"   Red Team vulnerabilities: {results['counts']['total_red_vulns']}")
print(f"   Blue Team findings:       {results['counts']['total_blue_findings']}")
print(f"   True Positives:           {results['counts']['true_positives']}")
print(f"   False Positives:          {results['counts']['false_positives']}")
print(f"   False Negatives:          {results['counts']['false_negatives']}")

In [ ]:
# Show detailed match analysis
print("\n🔍 DETAILED MATCH ANALYSIS:")
print("=" * 60)

if results.get('matches'):
    for match in results['matches']:
        print(f"\n✓ {match['match_type'].upper()} MATCH")
        print(f"  Red: {match.get('red_title', 'N/A')}")
        print(f"  Blue: {match.get('blue_title', 'N/A')}")
        print(f"  Confidence: {match.get('confidence', 0):.0%}")

if results['details'].get('false_negative_ids'):
    print(f"\n❌ EVADED DETECTION ({len(results['details']['false_negative_ids'])}):")
    for fn_id in results['details']['false_negative_ids']:
        print(f"   - {fn_id}")

if results['details'].get('false_positive_ids'):
    print(f"\n⚠️ FALSE ALARMS ({len(results['details']['false_positive_ids'])}):")
    for fp_id in results['details']['false_positive_ids']:
        print(f"   - {fp_id}")

In [ ]:
## 6. 💾 Save Results (Optional)

Save the game results for later analysis.

# Save results to output directory
output_dir = project_root / "output" / f"game_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
output_dir.mkdir(parents=True, exist_ok=True)

# Save code files
code_dir = output_dir / "code"
code_dir.mkdir(exist_ok=True)
for filename, content in red_result.files.items():
    (code_dir / filename).write_text(content)

# Save manifests and results
with open(output_dir / "red_manifest.json", "w") as f:
    json.dump({"vulnerabilities": red_result.vulnerabilities}, f, indent=2)

with open(output_dir / "blue_findings.json", "w") as f:
    json.dump({"findings": blue_result.findings}, f, indent=2)

with open(output_dir / "scoring.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"✓ Results saved to: {output_dir}")

In [ ]:
## 🎯 Summary

This notebook demonstrated a single adversarial game. Key takeaways:

- **Precision** tells you how accurate Blue Team's detections were (fewer false alarms = higher precision)
- **Recall** tells you how complete the detection was (fewer missed vulns = higher recall)  
- **F1 Score** balances both metrics
- **Evasion Rate** shows how successful Red Team was at hiding vulnerabilities

### Next Steps
- Try different difficulty levels (`easy`, `medium`, `hard`)
- Compare different models (Haiku vs Sonnet)
- Use `HYBRID` mode to combine LLM with static analysis tools
- Run batch experiments with `02_run_experiment.ipynb`

# Final summary
print("=" * 60)
print("🎮 GAME COMPLETE")
print("=" * 60)
print(f"\n🔴 Red Team injected {len(red_result.vulnerabilities)} vulnerabilities")
print(f"🔵 Blue Team found {len(blue_result.findings)} issues")
print(f"⚖️  F1 Score: {results['metrics']['f1_score']:.1%}")

if results['metrics']['f1_score'] >= 0.7:
    print("\n🏆 Blue Team wins! Strong detection performance.")
elif results['metrics']['evasion_rate'] >= 0.3:
    print("\n🏆 Red Team wins! Successfully evaded detection.")
else:
    print("\n🤝 Close match! Both teams performed well.")